In [1]:
# %pip show pedalboard
# %pip show numpy
%pip install soundfile

  Obtaining dependency information for soundfile from https://files.pythonhosted.org/packages/57/5e/70bdd9579b35003a489fc850b5047beeda26328053ebadc1fb60f320f7db/soundfile-0.13.1-py2.py3-none-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for cffi>=1.0 from https://files.pythonhosted.org/packages/b2/d5/da47df7004cb17e4955df6a43d14b3b4ae77737dff8bf7f8f333196717bf/cffi-1.17.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pycparser from https://files.pythonhosted.org/packages/13/a3/a812df4e2dd5696d1f351d58b8fe16a405b234ad2886a0dab9183fb78109/pycparser-2.22-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.4/479.4 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 27.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: 

In [2]:
# Generate Audio Data to train on
from pedalboard import *
from pedalboard.io import AudioFile

# Takes in a filename and pedalboard setting to apply
def createWithEffect(inputFile: str, outputPath: str, board: Pedalboard) -> str:
    # Make a Pedalboard object, containing multiple audio plugins:

    # Open an audio file for reading, just like a regular file:
    with AudioFile(inputFile) as f:

        # Open an audio file to write to:
        outputPath = f"{outputPath}/{inputFile.split('/')[-1][:-4]}_output.wav" 
        with AudioFile(outputPath, 'w', f.samplerate, f.num_channels) as o:

            # Read one second of audio at a time, until the file is empty:
            while f.tell() < f.frames:
                chunk = f.read(f.samplerate)

                # Run the audio through our pedalboard:
                effected = board(chunk, f.samplerate, reset=False)

                # Write the output to our output file:
                o.write(effected)

board = Pedalboard([
    Compressor(threshold_db=-50, ratio=25),
    Gain(gain_db=30),
    Chorus(),
    LadderFilter(mode=LadderFilter.Mode.HPF12, cutoff_hz=900),
    Phaser(),
    Reverb(room_size=0.25),
])

# createWithEffect('data/guitarset/00_BN1-129-Eb_comp_mix.wav', 'data/generated', board)

ModuleNotFoundError: No module named 'pedalboard'

In [ ]:
# Apply random effects on this module and store this in json
import os
import json
import random
import pedalboard
import soundfile as sf
from pedalboard import *

# Define input and output directories
input_dir = 'data/guitarset'
output_dir = 'data/generated'
metadata_file = os.path.join(output_dir, 'metadata.json')

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# List of available effects to choose from
effects_list = [
    Chorus(rate_hz=random.uniform(0.5, 5.0)),
    Distortion(gain_db=random.uniform(5, 30)),
    Reverb(room_size=random.uniform(0.2, 1.0)),
    Phaser(rate_hz=random.uniform(0.1, 2.0)),
    HighpassFilter(cutoff_frequency_hz=random.uniform(500, 5000)),
    LowpassFilter(cutoff_frequency_hz=random.uniform(500, 5000)),
    Compressor(threshold_db=random.uniform(-30, -10), ratio=random.uniform(2, 6)),
    Gain(gain_db=random.uniform(-10, 10)),
    # PitchShift(semitones=random.uniform(-12, 12)),
    Delay(delay_seconds=random.uniform(0.1, 1.0), feedback=random.uniform(0.1, 0.9)),
    Limiter(threshold_db=random.uniform(-10, 0)),
    # Overdrive(gain_db=random.uniform(5, 20)),
    # Tremolo(rate_hz=random.uniform(1, 10)),
    # AutoWah(),
]

N = 5  # Number of random variations per file
metadata = []

# Process each audio file in input directory
for file in os.listdir(input_dir):
    if file.endswith('.wav'):
        input_path = os.path.join(input_dir, file)
        audio, sample_rate = sf.read(input_path)
        
        for i in range(N):
            # Choose a random subset of effects
            num_effects = random.randint(1, 4)  # Apply 1 to 4 effects
            chosen_effects = random.sample(effects_list, num_effects)
            
            # Apply the effects
            board = Pedalboard(chosen_effects)
            processed_audio = board(audio, sample_rate)
            
            # Generate output filename
            output_filename = f"{os.path.splitext(file)[0]}_variant_{i}.wav"
            output_path = os.path.join(output_dir, output_filename)
            
            # Save processed audio
            sf.write(output_path, processed_audio, sample_rate)
            
            # Log metadata
            metadata.append({
                "original": input_path,
                "generated": output_path,
                "effects": [str(effect) for effect in chosen_effects]
            })

# Save metadata to JSON
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=4)

print(f"Processing complete. Metadata saved to {metadata_file}")


In [ ]:
# Separate audio into separate files to perform CNN


he
